# Labo 5 : Vector Embedding

> Authors : Stefani Massimo, Häffner Edwin

## Partie 1

In [3]:
# Installation de gensim et scipy en dernière version
!pip install gensim scipy

  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 6.5 MB/s eta 0:00:00m eta 0:00:010:01:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.1
    Uninstalling numpy-2.0.1:
      Successfully uninstalled numpy-2.0.1
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --u

In [4]:
#Installation du modèle entraîné sur Google News 
from gensim import downloader as api
w2v_vectors = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [11]:
#Chargement du modèle
from gensim.models import KeyedVectors
w2v_vectors = KeyedVectors.load_word2vec_format("~/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz", binary=True)


### Réponse aux questions 

>a. Quelle place en mémoire occupe le processus du notebook avec les vecteurs de mots ?

Sur mon OS Linux, je vois que le notebook prends 4.57 GiB en mémoire. Visual studio code en prends 8.52 GiB mais je ne pense pas que c'est lié ! 



In [13]:
# Question b
# Quelle est la dimension de l’espace vectoriel dans lequel les mots sont représentés ?

print(f"La dimension de l'espace vectoriel est de {w2v_vectors.vector_size} dimensions")


La dimension de l'espace vectoriel est de 300 dimensions


In [32]:
# Question c
# Quelle est la taille du vocabulaire connu du modèle ? Veuillez afficher cinq mots anglais qui sont dans le vocabulaire et deux qui ne le sont pas.

print(f"La taille du vocabulaire est de {len(w2v_vectors)} mots et voici des mots qui sont dans le vocabulaire (ou pas): ")

for word in ["natural", "language", "processing", "Yverdon", "school", "bumfuzzle", "taradiddle", "collywobbles", "Yverdon-Les-Bains", "Puidoux"]:
    if word in w2v_vectors :
        print(f"Connu : {word}")
    else :
        print(f"Inconnu : {word}")

        


La taille du vocabulaire est de 3000000 mots et voici des mots qui sont dans le vocabulaire (ou pas): 
Connu : natural
Connu : language
Connu : processing
Connu : Yverdon
Connu : school
Inconnu : bumfuzzle
Inconnu : taradiddle
Connu : collywobbles
Inconnu : Yverdon-Les-Bains
Inconnu : Puidoux


In [ ]:
# Question d 
# Quelle est la similarité entre les mots rabbit et carrot ? Veuillez rappeler comment on mesure les similarités entre deux mots grâce à leurs vecteurs.
print('La similarité entre \"rabbit\" et \"carrot\" est de : %.4f' % (w2v_vectors.similarity("rabbit", "carrot")))


La similarité entre "rabbit" et "carrot" est de : 0.3631


In [51]:
# Question e 
# Considérez au moins 5 paires de mots anglais, certains proches par leurs sens, d’autres plus éloignés. Pour chaque paire, calculez la similarité entre les deux mots. Veuillez indiquer si les similarités obtenues correspondent à vos intuitions sur la proximité des sens des mots.

pairs = [
    ('rabbit', 'animal'), # Un lapin EST un animal
    ('rabbit', 'hare'), # Le lièvre est un genre de lapin "sauvage"
    ('rabbit', 'bunny'),  # Version cute de lapin
    ('rabbit', 'easter'),    # lapin de Pâques
    ('rabbit', 'vertex'), #Rien à voir
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, w2v_vectors.similarity(w1, w2)))

'rabbit'	'animal'	0.53
'rabbit'	'hare'	0.61
'rabbit'	'bunny'	0.64
'rabbit'	'easter'	0.19
'rabbit'	'vertex'	0.07


> Question e : Discussion

- On voit que lapin et animal est assez similaire ce qui fais sens
- Lapin et lièvre sont encore plus similaire ce qui fais aussi sens
- Un peu bizarre que rabbit et bunny ne soient pas plus similaire que ça vu que c'est littéralement le même animal, juste que bunny est la forme plus colloquial. 
- Rabbit et easter ne sont pas similaire ce qui fait sens, mais il y a quand même un lien (lapin de Pâques donc la similitude n'est pas si basse)
- Enfin rabbit et vertex (sommet) n'ont vraiment rien à voir donc 0.07 n'est pas surprenant.

In [ ]:
# Question f 
# Pouvez-vous trouver des mots de sens opposés mais qui sont proches selon le modèle ?
w1 = "awful"
w2 = "good"
print(f'Similarité entre {w1} et {w2} %.4f' % (w2v_vectors.similarity(w1, w2)))


# Comment expliquez-vous cela ? Est-ce une qualité ou un défaut du modèle word2vec ?
# Je pense que awful et good peuvent être utilisé dans des même contexte, comme lorsqu'on a un article qui évalue des choses, on peut à la fois considérer quelque chose de bon 

Similarité entre awfully et good 0.3707
